In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
from handler import geozoom_handler
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from attn_model2 import *
from fc_net import *
from helpers import *
from utils2 import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 5)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

5 municipalities.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNetBinary(in_channels = 512, h = 7, w = 7, batch_size = 1, resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)



butler = geozoom_handler(attn_model, fc_model, device, criterion, attn_optimizer, fc_optimizer, plot = False, v = True)

In [5]:
BATCH_SIZE = 1
SPLIT = .90

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [6]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape, o)

torch.Size([1, 3, 2083, 1503]) tensor([9.], dtype=torch.float64)
torch.Size([1, 3, 476, 619]) tensor([536.], dtype=torch.float64)
torch.Size([1, 3, 1987, 1725]) tensor([6205.], dtype=torch.float64)
torch.Size([1, 3, 535, 538]) tensor([299.], dtype=torch.float64)


In [7]:
501 * .70

350.7

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1758.019072175026
Loss thresholds for training:  [0.0, 251.1455817392894, 502.2911634785788, 753.4367452178683, 1004.5823269571576, 1255.727908696447, 1506.8734904357366]
Starting from threshold:  6  with value:  1506.8734904357366


Epoch:  1
  Training Loss:  1735.269968032837


Epoch:  2
  Training Loss:  1644.836441040039


Epoch:  3
  Training Loss:  1680.7976684570312


Epoch:  4
  Training Loss:  1674.3608646392822


Epoch:  5
  Training Loss:  1538.979995727539


Epoch:  6
  Training Loss:  1516.752965927124


Epoch:  7
  Training Loss:  1518.8199672698975


Epoch:  8
  Training Loss:  1516.9652490615845


Epoch:  9
  Training Loss:  1434.076452255249
  Moving to threshold:  5   |  Next loss benchmark:  1255.727908696447


484031047
old image size:  (535, 538)
new image size:  374 376


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




484019040
old image size:  (2083, 1503)
new image size:  1458 1052


484014057
old image size:  (476, 619)
new image size:  333 433


484020183
old image size:  (1987, 1725)
new image size:  1390 1207


Epoch:  10
  Training Loss:  1657.4707374572754


Epoch:  11
  Training Loss:  1599.37548828125


Epoch:  12
  Training Loss:  1560.3953647613525


Epoch:  13
  Training Loss:  1555.0896434783936


Epoch:  14
  Training Loss:  1498.7398624420166


Epoch:  15
  Training Loss:  1501.0618228912354


Epoch:  16
  Training Loss:  1512.9911937713623


Epoch:  17
  Training Loss:  1445.7095069885254


Epoch:  18
  Training Loss:  1456.145975112915


Epoch:  19
  Training Loss:  1412.206979751587


Epoch:  20
  Training Loss:  1367.6400737762451


Epoch:  21
  Training Loss:  1387.0708312988281


Epoch:  22
  Training Loss:  1285.8391342163086


Epoch:  23
  Training Loss:  1119.8451347351074
  Moving to threshold:  4   |  Next loss benchmark:  1004.5823269571576


484014057
old image size:  

In [9]:
butler.image_sizes

{'484024023': [(0, 1126, 0, 1507),
  (168, 956, 226, 1280),
  (117, 668, 157, 894),
  (82, 467, 110, 625),
  (57, 326, 77, 437),
  (22, 246, 67, 291)],
 '484020565': [(0, 470, 0, 395), (70, 399, 59, 335), (52, 276, 26, 250)],
 '484021030': [(0, 217, 0, 124)],
 '484021171': [(0, 195, 0, 170)]}

In [10]:
butler.threshold_weights.keys()

dict_keys([6, 5, 4, 3, 2, 1, 'fc'])

In [11]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape)

torch.Size([1, 3, 1126, 1507])
torch.Size([1, 3, 470, 395])
torch.Size([1, 3, 217, 124])
torch.Size([1, 3, 195, 170])


In [12]:
641 * .70

448.7

In [ ]:
263 - 

In [8]:
for i, o in train_dl:
    
    muni_id = i[0].split("/")[4]
    
    plt.imshow(butler.prep_input(i).detach().cpu()[0].permute(1,2,0))
    plt.title(butler.image_sizes[muni_id])
    plt.savefig(f"test{muni_id}.png")
    
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [9]:
# # for i in range(50):
# while handler.threshold_index > 0:

#     for impath, output in train_dl:

#         # Prep the input and pass it to the trainer (this could easily be done in one step eventually if ya want)
#         input = handler.prep_input(impath)
#         handler.train(input, output)
    
#     handler.end_epoch(train_dl, val_dl = None)

In [8]:
handler.image_sizes

{'484002003': (0, 224, 0, 224),
 '484016045': (0, 224, 0, 224),
 '484020489': (0, 224, 0, 224),
 '484020254': (0, 224, 0, 224)}

In [19]:
butler.threshold_weights.keys()

dict_keys([6, 5, 4, 3, 2, 1, 'fc'])

In [10]:
for i, o in val_dl:
    print(i, o)

('../pooling/data/MEX2/484020535/pngs/484020535_2010_1_box484020535_MAY.png',) tensor([89.], dtype=torch.float64)


In [13]:
def run_validation(attn_model, fc_model, input, weights_dict, plot = False):
    
    muni_id = input[0].split("/")[4]
    cur_image = load_inputs(input[0])    
    
    for k in weights_dict.keys():
                
        if k != 'fc':
            
            model = attn_model
            model.load_state_dict(weights_dict[k])
            model.eval()
            IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
            gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda()) 
            cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)
            
            if plot == True:
            
                plot_gradcam(gradcam)
                plt.savefig(f"threshold{k}_muni{muni_id}.png")
                plt.clf()
            
        
        if k == 'fc':
            
            model = fc_model
            model.load_state_dict(weights_dict[k])
            model.eval()
            return model(cur_image.cuda()).item()
            

        
for i, o in train_dl:

    print(round(run_validation(attn_model, fc_model, i, butler.threshold_weights), 2), "     ", o.item())
    

201.76       196.0
201.08       4.0
273.5       606.0
4416.25       6521.0


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

...

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>